In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import sys
sys.path.insert(0, '..')
import warnings
warnings.filterwarnings('ignore')

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['font.size'] = 12
import pandas as pd
import xarray as xr

import covid19

In [ ]:
data_italy_path = 'dpc-covid19-ita-regioni.csv'

data_italy = covid19.data.read_dpc(data_italy_path)

data_italy = data_italy.assign_coords({'location': ('location', [l.partition(' / ')[2] for l in data_italy.location.values])})
data_italy = data_italy.drop(['country', 'lat', 'lon', 'state_region'])
data_italy = xr.merge([data_italy, data_italy.sum('location').expand_dims(location=['Italia'])])
data_italy = data_italy.sortby(data_italy.population)

for col in ['deaths', 'confirmed', 'tests', 'tested']:
    data_italy['daily_' + col] = data_italy[col].diff('time')
    data_italy['daily_' + col + '7'] = data_italy['daily_' + col].rolling({'time': 7}).sum()

data_italy

In [ ]:
data_pm_italy = (data_italy / data_italy.population * 1_000_000).drop(['population'])
data_pm_italy['positive_test_rate'] = data_pm_italy['daily_confirmed'] / data_pm_italy['daily_tests']
data_pm_italy['positive_test_rate7'] = data_pm_italy['daily_confirmed7'] / data_pm_italy['daily_tests7']
data_pm_italy

In [ ]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=20, metadata=dict(artist='Alessandro Amici'), bitrate=1800)

In [ ]:
data = data_pm_italy.sel(time=slice('2020-05-01', None))
data['positive_test_rate7'] = xr.where(data['positive_test_rate7'] < 0.001, 0.001, data['positive_test_rate7'])

ani = covid19.plot.animate_scatter(data['daily_confirmed7'], data['positive_test_rate7'], xlim=(0.9, 1000 / 0.9), ylim=(0.0009, 0.10 / 0.9), tail=12, **{
    'xscale': 'log', 'yscale': 'log',
    'xlabel': 'nuovi casi negli ultimi 7 giorni per milione di abitanti', 'ylabel': 'percentuale di test positivi negli ultimi 7 giorni',
})

In [ ]:
# ani.save('covid19-italy.mp4', writer=writer)

In [ ]:
data = data_pm_italy.sel(time=slice('2020-09-15', None))

ani = covid19.plot.animate_scatter(data['daily_confirmed7'], data['positive_test_rate7'], xlim=(40 * 0.9, 1000 / 0.9), ylim=(0.004 * 0.9, 0.10 / 0.9), tail=18, freq='4h', **{
    'xscale': 'log', 'yscale': 'log',
    'xlabel': 'nuovi casi negli ultimi 7 giorni per milione di abitanti', 'ylabel': 'percentuale di test positivi negli ultimi 7 giorni',
})

In [ ]:
ani.save('covid19-italy-small.mp4', writer=writer)

In [ ]:
data = data_pm_italy.sel(time=slice('2020-09-15', None))

ani = covid19.plot.animate_scatter(data['daily_confirmed7'] / 7, data['positive_test_rate7'], xlim=(0, 660), ylim=(0, 0.22), tail=24, freq='4h', **{
    'xscale': 'linear', 'yscale': 'linear',
    'xlabel': 'nuovi casi per milione di abitanti (media a 7 giorni)', 'ylabel': 'percentuale di test positivi negli ultimi 7 giorni',
})

In [ ]:
ani.save('covid19-italy-small-linear.mp4', writer=writer)

In [ ]:
lut_path = 'UID_ISO_FIPS_LookUp_Table.csv'
data_world_deaths_path = 'time_series_covid19_deaths_global.csv'
data_world_confirmed_path = 'time_series_covid19_confirmed_global.csv'
data_world = covid19.data.read_jhu_global(data_world_deaths_path, lut_path=lut_path)
data_world['confirmed'] = covid19.data.read_jhu_global(data_world_confirmed_path)['deaths']

data_world = data_world.sortby(data_world.population)

for col in ['deaths', 'confirmed']:
    data_world['daily_' + col] = data_world[col].diff('time')
    data_world['daily_' + col + '7'] = data_world['daily_' + col].rolling({'time': 7}).sum()

data_world

In [ ]:
data_pm_world = (data_world / data_world.population * 1_000_000).drop(['population'])
data_pm_world

In [ ]:
data = data_pm_world.sel(time=slice('2020-03-09', None), location=data_world['deaths'].isel(time=-1) > 1000)
data['daily_deaths7'] = xr.where(data['daily_deaths7'] < 0.31, 0.31, data['daily_deaths7'])

ani = covid19.plot.animate_scatter(data['deaths'], data['daily_deaths7'], xlim=(9, 1500), ylim=(0.3, 200 / 0.9), **{
    'xscale': 'log', 'yscale': 'log',
    'xlabel': 'decessi totali per milione di abitanti', 'ylabel': 'decessi negli ultimi 7 giorni per milione di abitanti',
})

In [ ]:
ani.save('covid19-world.mp4', writer=writer)

In [ ]:
data = data_pm_world.sel(time=slice('2020-03-09', None), location=data_world['deaths'].isel(time=-1) > 1000)
data = data.sel(location=['Italy', 'Israel', 'France', 'Spain', 'United Kingdom', 'US', 'Germany', 'Netherlands', 'Belgium', 'Sweden', 'Switzerland'])
data['daily_deaths7'] = xr.where(data['daily_deaths7'] < 0.31, 0.31, data['daily_deaths7'])

ani = covid19.plot.animate_scatter(data['daily_confirmed7'], data['daily_deaths7'], xlim=(9, 5500), ylim=(0.3, 200 / 0.9), tail=12, **{
    'xscale': 'log', 'yscale': 'log',
    'xlabel': 'nuovi casi negli ultimi 7 giorni per milione di abitanti', 'ylabel': 'decessi negli ultimi 7 giorni per milione di abitanti',
})

In [ ]:
ani.save('covid19-world2.mp4', writer=writer)